In [9]:
%%capture
!pip install import_ipynb --no-cache
import import_ipynb
m = __import__("Methods")

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy.stats import boxcox
import pandas as pd
import zipfile
import urllib.request
import os

In [7]:
rna_seq = dict()
phenotype = dict()
meta = dict()

# RR-1 Data

In [10]:
# Dataset identifiers
datasets = ["47", "48", "137", "168"]

# --- Load Metadata ---
for ds in datasets:
    meta_path = f"../meta_data/OSD-{ds}_metadata_OSD-{ds}-ISA/s_OSD-{ds}.txt"
    if os.path.exists(meta_path):
        print(f"Loading metadata for OSD-{ds}...")
        meta[ds] = pd.read_csv(meta_path, sep="\t", header=0)

# --- Load RNA-seq Data ---
for ds in datasets:
    rna_path = f"../raw_data/GLDS-{ds}_rna_seq_Normalized_Counts.csv"
    if os.path.exists(rna_path):
        print(f"Loading RNA-seq data for OSD-{ds}...")
        rna_seq[ds] = pd.read_csv(rna_path)

# --- Load Phenotype Data ---
pheno_sources = {
    "47": "LSDS-29_Histology_OSD_47_Histology_TRANSFORMED.csv",
    "48": "LSDS-2_Histology_OSD_48_Histology_TRANSFORMED.csv",
    "137": "LSDS-28_Histology_OSD_137_Histology_TRANSFORMED.csv"
}

for ds, fname in pheno_sources.items():
    pheno_path = f"../raw_data/{fname}"
    if os.path.exists(pheno_path):
        print(f"Loading phenotype data for OSD-{ds}...")
        phenotype[ds] = pd.read_csv(pheno_path)

# --- Filter RNA-seq Data ---
for ds in rna_seq:
    print(f"Filtering RNA-seq data for OSD-{ds}...")

    df = rna_seq[ds].copy()

    if ds == "168":
        # Convert index to 'Unnamed: 0' column for compatibility with filter_data
        df = df.reset_index().rename(columns={"index": "Unnamed: 0"})

    rna_seq[ds] = m.filter_data(df, dropnans=True, dropgenes=True, droplowcvs=0.5, droplowcount=10)


# --- Transpose RNA-seq Data ---
for ds in rna_seq:
    print(f"Transposing RNA-seq data for OSD-{ds}...")
    rna_seq[ds] = m.transpose_df(rna_seq[ds], cur_index_col="Unnamed: 0", new_index_col="Sample name")


# --- Align RNA-seq and Phenotype by Common Samples (47, 48) ---
for ds in ["47", "48", "137"]:
    if ds in rna_seq and ds in phenotype:
        print(f"Aligning RNA-seq and phenotype data for OSD-{ds}...")
        rna = rna_seq[ds]
        pheno = phenotype[ds]

        common = set(rna["Sample name"]) & set(pheno["Sample name"])
        rna_common = rna[rna["Sample name"].isin(common)].copy()
        pheno_common = pheno[pheno["Sample name"].isin(common)].copy()
        pheno_common = pheno_common[["Sample name", "ORO Positivity (%)"]]

        rna_common.reset_index(drop=True, inplace=True)
        pheno_common.reset_index(drop=True, inplace=True)

        print(f"Saving aligned data for OSD-{ds}...")
        rna_common.to_csv(f"../x_variables/rna_seq-{ds}.csv", index=False)
        pheno_common.to_csv(f"../y_variables/pheno-{ds}.csv", index=False)

# --- Special Case: phenotype["137"] aligned with rna_seq["168"] ---
if "168" in rna_seq and "137" in phenotype:
    print("Aligning RNA-seq data from OSD-168 with phenotype data from OSD-137...")
    rna = rna_seq["168"]
    pheno = phenotype["137"]

    common = set(rna["Sample name"]) & set(pheno["Sample name"])
    rna_common = rna[rna["Sample name"].isin(common)].copy()
    pheno_common = pheno[pheno["Sample name"].isin(common)].copy()
    pheno_common = pheno_common[["Sample name", "ORO Positivity (%)"]]

    rna_common.reset_index(drop=True, inplace=True)
    pheno_common.reset_index(drop=True, inplace=True)

    print("Saving aligned data for OSD-168 (RNA) and OSD-137 (phenotype)...")
    rna_common.to_csv("../x_variables/rna_seq-168.csv", index=False)
    pheno_common.to_csv("../y_variables/pheno-168.csv", index=False)


Loading metadata for OSD-47...
Loading metadata for OSD-48...
Loading metadata for OSD-137...
Loading metadata for OSD-168...
Loading RNA-seq data for OSD-47...
Loading RNA-seq data for OSD-48...
Loading RNA-seq data for OSD-137...
Loading RNA-seq data for OSD-168...
Loading phenotype data for OSD-47...
Loading phenotype data for OSD-48...
Loading phenotype data for OSD-137...
Filtering RNA-seq data for OSD-47...
Number of protein coding genes: 15059
2520
Filtering RNA-seq data for OSD-48...
Number of protein coding genes: 15991
5244
Filtering RNA-seq data for OSD-137...
Number of protein coding genes: 15567
3004
Filtering RNA-seq data for OSD-168...
Number of protein coding genes: 18551
6661
Transposing RNA-seq data for OSD-47...
Transposing RNA-seq data for OSD-48...
Transposing RNA-seq data for OSD-137...
Transposing RNA-seq data for OSD-168...
Aligning RNA-seq and phenotype data for OSD-47...
Saving aligned data for OSD-47...
Aligning RNA-seq and phenotype data for OSD-48...
Saving

In [11]:
# --- Identify common genes across all RNA-seq datasets ---
print("Identifying common genes across datasets...")
common_genes = None

for ds, df in rna_seq.items():
    gene_columns = set(df.columns) - {"Sample name"}  # exclude metadata column
    if common_genes is None:
        common_genes = gene_columns
    else:
        common_genes &= gene_columns  # intersection

common_genes = sorted(list(common_genes))  # sort for consistent order

print(f"Number of common genes across datasets: {len(common_genes)}")

# --- Subset each RNA-seq dataset to common genes and concatenate ---
rna_combined = []

for ds, df in rna_seq.items():
    print(f"Subsetting and appending OSD-{ds}...")
    subset = df[["Sample name"] + common_genes].copy()
    subset["Dataset"] = f"OSD-{ds}"  # optional: track source
    rna_combined.append(subset)

# Combine all into a single DataFrame
rna_all = pd.concat(rna_combined, axis=0, ignore_index=True)

Identifying common genes across datasets...
Number of common genes across datasets: 1354
Subsetting and appending OSD-47...
Subsetting and appending OSD-48...
Subsetting and appending OSD-137...
Subsetting and appending OSD-168...


In [17]:
# --- Combine all phenotype data into one ---
print("Combining phenotype data across datasets...")

pheno_combined = []

# Standard datasets: 47, 48
for ds in ["47", "48", "137"]:
    if ds in phenotype:
        df = phenotype[ds][["Sample name", "ORO Positivity (%)"]].copy()
        df["Dataset"] = f"OSD-{ds}"
        pheno_combined.append(df)

# Combine all into a single DataFrame
pheno_all = pd.concat(pheno_combined, axis=0, ignore_index=True)

Combining phenotype data across datasets...


In [18]:
# Strip whitespace to ensure matches
rna_all["Sample name"] = rna_all["Sample name"].str.strip()
pheno_all["Sample name"] = pheno_all["Sample name"].str.strip()

# Find common samples
common_samples = set(rna_all["Sample name"]) & set(pheno_all["Sample name"])
print(f"Number of common samples: {len(common_samples)}")

# Filter both datasets
rna_all = rna_all[rna_all["Sample name"].isin(common_samples)].reset_index(drop=True)
pheno_all = pheno_all[pheno_all["Sample name"].isin(common_samples)].reset_index(drop=True)

Number of common samples: 41


In [20]:
def extract_environment(sample_name):
    parts = sample_name.split("_")
    if len(parts) >= 4:
        env = parts[3]
        return env if env == "FLT" else "GC"
    return "UNKNOWN"

# Apply to both datasets
rna_all["Environment"] = rna_all["Sample name"].apply(extract_environment)
pheno_all["Environment"] = pheno_all["Sample name"].apply(extract_environment)

# Apply to both datasets
rna_all["Environment"] = rna_all["Sample name"].apply(extract_environment)
pheno_all["Environment"] = pheno_all["Sample name"].apply(extract_environment)

In [21]:
rna_all.to_csv("../x_variables/rna_seq-RR1.csv", index=False)
print(f"Combined RNA-seq data saved with {rna_all.shape[0]} samples and {rna_all.shape[1] - 2} genes.")
pheno_all.to_csv("../y_variables/pheno_seq-RR1.csv", index=False)
print(f"Combined phenotype data saved with {pheno_all.shape[0]} samples.")

Combined RNA-seq data saved with 41 samples and 1355 genes.
Combined phenotype data saved with 41 samples.


In [22]:
# --- Merge RNA-seq and phenotype datasets on Sample name ---
combined_df = pd.merge(rna_all, pheno_all, on="Sample name", suffixes=("_rna", "_pheno"))

# Optional: reordering columns
# Move metadata columns to the front
meta_cols = ["Sample name", "Dataset_rna", "Environment_rna", "ORO Positivity (%)"]
gene_cols = [col for col in combined_df.columns if col not in meta_cols]
combined_df = combined_df[meta_cols + gene_cols]

combined_df.drop(columns=["Dataset_rna", "Dataset_pheno", "Environment_pheno"], axis=1, inplace=True)
combined_df.rename(columns={"Environment_rna": "Environment"}, inplace=True)

# --- Save the combined dataset ---
combined_df.to_csv("../ml_data/RR1-combined.csv", index=False)
print(f"Combined dataset saved with {combined_df.shape[0]} samples and {combined_df.shape[1]} columns.")


Combined dataset saved with 41 samples and 1357 columns.


In [44]:
combined_df.columns

Index(['Sample name', 'Environment', 'ORO Positivity (%)',
       'ENSMUSG00000000094', 'ENSMUSG00000000365', 'ENSMUSG00000000402',
       'ENSMUSG00000000416', 'ENSMUSG00000000562', 'ENSMUSG00000000627',
       'ENSMUSG00000000794',
       ...
       'ENSMUSG00000116378', 'ENSMUSG00000116461', 'ENSMUSG00000116594',
       'ENSMUSG00000116780', 'ENSMUSG00000117081', 'ENSMUSG00000117286',
       'ENSMUSG00000117748', 'ENSMUSG00000117874', 'Dataset_pheno',
       'Environment_pheno'],
      dtype='object', length=1359)

In [16]:
df = pd.read_csv("../ml_data/RR1-combined.csv")

non_string_df = df.select_dtypes(exclude=['string'])

# Get the column names
non_string_columns = non_string_df.columns
non_string_columns

Index(['Sample name', 'Environment', 'ORO Positivity (%)',
       'ENSMUSG00000000094', 'ENSMUSG00000000365', 'ENSMUSG00000000402',
       'ENSMUSG00000000416', 'ENSMUSG00000000562', 'ENSMUSG00000000627',
       'ENSMUSG00000000794',
       ...
       'ENSMUSG00000115232', 'ENSMUSG00000115958', 'ENSMUSG00000116378',
       'ENSMUSG00000116461', 'ENSMUSG00000116594', 'ENSMUSG00000116780',
       'ENSMUSG00000117081', 'ENSMUSG00000117286', 'ENSMUSG00000117748',
       'ENSMUSG00000117874'],
      dtype='object', length=1357)